In [2]:
path_to_yolo = '../repos/yolov5/'

import argparse
import sys
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn


In [28]:
YOLO_PATH = Path(path_to_yolo).absolute()
sys.path.append(YOLO_PATH.as_posix())  # add yolov5/ to path

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages, letterbox
from utils.general import check_img_size, check_requirements, check_imshow, colorstr, non_max_suppression, \
    apply_classifier, scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path, save_one_box
from utils.plots import colors, plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

In [39]:
def run(weights='yolov5s.pt',  # model.pt path(s)
        source='data/images',  # file/dir/URL/glob, 0 for webcam
        imgsz=640,  # inference size (pixels)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        update=False,  # update all models
        project='runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        ):

    # Initialize
    device = select_device(device)
    half &= device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check image size
    names = model.module.names if hasattr(model, 'module') else model.names  # get class names
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet50', n=2)  # initialize
        modelc.load_state_dict(torch.load('resnet50.pt', map_location=device)['model']).to(device).eval()

    # Set Dataloader
    dataset = LoadImages(source, img_size=imgsz, stride=stride)
    crop_count=0
    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        
        #Obtain mask path
        basename  = os.path.basename(path)
        img_num = basename.split('_')[1]
        mask_path = '../datasets/segmentation/labels/test_localization_%.8i_target.png' % int(img_num)
        print(mask_path)
        mask0 = cv2.imread(mask_path)  # BGR
        
        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy()
            maskc = mask0.copy()
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    crop_count+=1
                    c = int(cls)  # integer class
                    label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                    plot_one_box(xyxy, im0, label=label, color=colors(c, True), line_thickness=line_thickness)
                    
                    #Save cropped image
                    save_one_box(xyxy, imc, file='../datasets/segmentation/crops/images/%.8i.jpg'%crop_count, BGR=True)
                    #Save cropped mask
                    save_one_box(xyxy, maskc, file='../datasets/segmentation/crops/label/%.8i.jpg'%crop_count, BGR=True)
                    
            # Print time (inference + NMS)
            print(f'{s}Done. ({t2 - t1:.3f}s)')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond


    print(f'Done. ({time.time() - t0:.3f}s)')

In [ ]:
model_weights = '../repos/yolov5/runs/train/exp6/weights/best.pt'
source = '../datasets/segmentation/images/'
run(weights=model_weights, source=source,
    imgsz=1280,  # inference size (pixels)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='0',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=True,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=True)

image 1/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000000_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000000_target.png
Done. (0.066s)
image 2/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000001_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000001_target.png
Done. (0.070s)
image 3/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000002_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000002_target.png
Done. (0.066s)
image 4/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000003_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000003_target.png
Done. (0.061s)
image 5/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000004_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000004_t

Done. (0.071s)
image 41/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000040_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000040_target.png
Done. (0.066s)
image 42/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000041_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000041_target.png
Done. (0.062s)
image 43/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000042_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000042_target.png
Done. (0.063s)
image 44/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000043_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000043_target.png
Done. (0.063s)
image 45/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000044_pre_disaster.png: ../datasets/segmentation/labels/test_loc

Done. (0.065s)
image 81/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000080_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000080_target.png
Done. (0.069s)
image 82/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000081_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000081_target.png
Done. (0.070s)
image 83/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000082_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000082_target.png
Done. (0.069s)
image 84/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000083_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000083_target.png
Done. (0.071s)
image 85/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000084_pre_disaster.png: ../datasets/segmentation/labels/test_loc

Done. (0.068s)
image 121/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000120_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000120_target.png
Done. (0.066s)
image 122/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000121_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000121_target.png
Done. (0.068s)
image 123/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000122_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000122_target.png
Done. (0.067s)
image 124/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000123_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000123_target.png
Done. (0.071s)
image 125/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000124_pre_disaster.png: ../datasets/segmentation/labels/tes

Done. (0.061s)
image 162/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000161_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000161_target.png
Done. (0.068s)
image 163/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000162_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000162_target.png
Done. (0.068s)
image 164/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000163_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000163_target.png
Done. (0.067s)
image 165/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000164_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00000164_target.png
Done. (0.069s)
image 166/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00000165_pre_disaster.png: ../datasets/segmentation/labels/tes

...

Done. (0.073s)
image 2735/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002734_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002734_target.png
Done. (0.063s)
image 2736/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002735_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002735_target.png
Done. (0.068s)
image 2737/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002736_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002736_target.png
Done. (0.061s)
image 2738/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002737_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002737_target.png
Done. (0.061s)
image 2739/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002738_pre_disaster.png: ../datasets/segmentation/label

Done. (0.064s)
image 2776/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002775_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002775_target.png
Done. (0.067s)
image 2777/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002776_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002776_target.png
Done. (0.069s)
image 2778/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002777_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002777_target.png
Done. (0.069s)
image 2779/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002778_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002778_target.png
Done. (0.068s)
image 2780/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002779_pre_disaster.png: ../datasets/segmentation/label

Done. (0.070s)
image 2816/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002815_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002815_target.png
Done. (0.068s)
image 2817/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002816_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002816_target.png
Done. (0.068s)
image 2818/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002817_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002817_target.png
Done. (0.068s)
image 2819/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002818_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002818_target.png
Done. (0.068s)
image 2820/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002819_pre_disaster.png: ../datasets/segmentation/label

Done. (0.069s)
image 2857/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002856_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002856_target.png
Done. (0.068s)
image 2858/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002857_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002857_target.png
Done. (0.064s)
image 2859/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002858_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002858_target.png
Done. (0.064s)
image 2860/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002859_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002859_target.png
Done. (0.063s)
image 2861/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002860_pre_disaster.png: ../datasets/segmentation/label

Done. (0.068s)
image 2898/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002897_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002897_target.png
Done. (0.067s)
image 2899/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002898_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002898_target.png
Done. (0.067s)
image 2900/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002899_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002899_target.png
Done. (0.068s)
image 2901/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002900_pre_disaster.png: ../datasets/segmentation/labels/test_localization_00002900_target.png
Done. (0.068s)
image 2902/9168 /home/francisco/Work/xplorespace/notebooks/../datasets/segmentation/images/test_00002901_pre_disaster.png: ../datasets/segmentation/label

In [26]:
from tqdm import tqdm
import os
import numpy as np
import tiffile
import json
import shapely.wkt
from PIL import Image, ImageDraw

folders = ['../datasets/geotiffs/tier1/','../datasets/geotiffs/tier3/']
#test_folder = '../datasets/geotiffs/test/'
destination_folder = '../datasets/segmentation/'

images = []
for folder in folders:
    images += [os.path.join(folder+'images', img_name) for img_name in os.listdir(folder+'images') if 'post' in img_name]

for i, post_img_path in tqdm(enumerate(images), desc="Processing Dataset"):
    basename = os.path.basename(post_img_path)
    disaster, img_num, _,_ = basename.split('_')
    test_folder = post_img_path.split('images')[0]

    pre_img_path = os.path.join(os.path.dirname(post_img_path), "%s_%s_pre_disaster.tif" % (disaster, img_num))

    pre_lbl_path = os.path.join(test_folder + 'labels', "%s_%s_pre_disaster.json" % (disaster, img_num))
    post_lbl_path = os.path.join(test_folder + 'labels', "%s_%s_post_disaster.json"% (disaster, img_num))

    with open(post_lbl_path, 'r') as infile:
        post_lbl = json.load(infile)
    with open(pre_lbl_path, 'r') as infile:
        pre_lbl = json.load(infile)

    damage_polys = {}
    polys = []
    for feat in pre_lbl['features']['xy']:
        #Check if feature is a building
        if feat['properties']['feature_type'] == 'building':
            poly_pre = shapely.wkt.loads(feat['wkt'])
            polys.append(poly_pre)

    for feat in post_lbl['features']['xy']:
        #Check if feature is a building
        if feat['properties']['feature_type'] == 'building':
            uid = feat["properties"]["uid"]
            damage = feat["properties"]["subtype"]
            if damage != 'un-classified':
                poly_post = shapely.wkt.loads(feat['wkt'])
                damage_polys[uid] = []
                damage_polys[uid].append(poly_post)
                damage_polys[uid].append(damage)
    try:
        pre_tif = tiffile.imread(pre_img_path).copy()
        #post_tif = tiffile.imread(post_img_path).copy()

        pre_tif = cv2.cvtColor(np.uint8(pre_tif), cv2.COLOR_RGB2BGR)
        #post_tif = cv2.cvtColor(np.uint8(post_tif), cv2.COLOR_RGB2BGR)

        loc_mask = create_loc_mask(pre_tif, polys, evaluate=True)
        #cls_mask = create_cls_mask(post_tif, damage_polys, evaluate=True)


        cv2.imwrite(destination_folder+'images/test_%.8i_pre_disaster.png'% i, pre_tif)
        #cv2.imwrite(destination_folder+'images/test_%.8i_post_disaster.png'% i, post_tif)

        cv2.imwrite(destination_folder+'labels/test_localization_%.8i_target.png' % i, loc_mask)
        #cv2.imwrite(destination_folder+'labels/test_damage_%.8i_target.png' % i, cls_mask)
    except Exception as e:
        print(e)
        pass



Processing Dataset: 0it [00:00, ?it/s]
Processing Dataset: 3it [00:00, 20.51it/s]
Processing Dataset: 6it [00:00, 20.73it/s]
Processing Dataset: 8it [00:00, 18.75it/s]
Processing Dataset: 10it [00:00, 13.59it/s]
Processing Dataset: 13it [00:00, 15.31it/s]
Processing Dataset: 15it [00:00, 14.81it/s]
Processing Dataset: 18it [00:01, 15.81it/s]
Processing Dataset: 20it [00:01, 16.67it/s]
Processing Dataset: 22it [00:01, 16.81it/s]
Processing Dataset: 25it [00:01, 17.97it/s]
Processing Dataset: 28it [00:01, 18.93it/s]
Processing Dataset: 31it [00:01, 18.82it/s]
Processing Dataset: 34it [00:01, 18.87it/s]
Processing Dataset: 37it [00:02, 20.61it/s]
Processing Dataset: 40it [00:02, 19.69it/s]
Processing Dataset: 43it [00:02, 20.61it/s]
Processing Dataset: 46it [00:02, 20.60it/s]
Processing Dataset: 49it [00:02, 21.65it/s]
Processing Dataset: 52it [00:02, 22.15it/s]
Processing Dataset: 55it [00:02, 20.82it/s]
Processing Dataset: 58it [00:03, 21.26it/s]
Processing Dataset: 61it [00:03, 21.33i

Processing Dataset: 442it [00:23, 17.88it/s]
Processing Dataset: 444it [00:23, 18.19it/s]
Processing Dataset: 447it [00:23, 19.23it/s]
Processing Dataset: 450it [00:24, 19.60it/s]
Processing Dataset: 453it [00:24, 19.12it/s]
Processing Dataset: 455it [00:24, 19.29it/s]
Processing Dataset: 457it [00:24, 17.22it/s]
Processing Dataset: 459it [00:24, 12.79it/s]
Processing Dataset: 462it [00:24, 14.76it/s]
Processing Dataset: 465it [00:25, 16.36it/s]
Processing Dataset: 467it [00:25, 16.70it/s]
Processing Dataset: 470it [00:25, 18.08it/s]
Processing Dataset: 472it [00:25, 18.38it/s]
Processing Dataset: 474it [00:25, 18.04it/s]
Processing Dataset: 476it [00:25, 18.19it/s]
Processing Dataset: 478it [00:25, 17.59it/s]
Processing Dataset: 481it [00:25, 18.60it/s]
Processing Dataset: 484it [00:26, 19.34it/s]
Processing Dataset: 487it [00:26, 19.91it/s]
Processing Dataset: 490it [00:26, 19.64it/s]
Processing Dataset: 493it [00:26, 19.97it/s]
Processing Dataset: 496it [00:26, 18.68it/s]
Processing

Processing Dataset: 818it [00:45, 16.96it/s]
Processing Dataset: 820it [00:45, 17.17it/s]
Processing Dataset: 823it [00:45, 17.98it/s]
Processing Dataset: 825it [00:45, 17.96it/s]
Processing Dataset: 827it [00:46, 18.11it/s]
Processing Dataset: 829it [00:46, 17.94it/s]
Processing Dataset: 831it [00:46, 12.89it/s]
Processing Dataset: 833it [00:46, 14.38it/s]
Processing Dataset: 835it [00:46, 15.55it/s]
Processing Dataset: 837it [00:46, 16.10it/s]
Processing Dataset: 840it [00:46, 16.36it/s]
Processing Dataset: 842it [00:47, 16.79it/s]
Processing Dataset: 844it [00:47, 17.23it/s]
Processing Dataset: 846it [00:47, 17.30it/s]
Processing Dataset: 848it [00:47, 17.15it/s]
Processing Dataset: 850it [00:47, 17.55it/s]
Processing Dataset: 852it [00:47, 16.76it/s]
Processing Dataset: 854it [00:47, 17.01it/s]
Processing Dataset: 856it [00:47, 17.08it/s]
Processing Dataset: 858it [00:47, 17.35it/s]
Processing Dataset: 860it [00:48, 17.30it/s]
Processing Dataset: 862it [00:48, 17.62it/s]
Processing

Processing Dataset: 1167it [01:06, 16.23it/s]
Processing Dataset: 1169it [01:06, 16.87it/s]
Processing Dataset: 1171it [01:06, 16.37it/s]
Processing Dataset: 1173it [01:06, 16.40it/s]
Processing Dataset: 1175it [01:06, 17.08it/s]
Processing Dataset: 1177it [01:06, 16.75it/s]
Processing Dataset: 1179it [01:06, 16.92it/s]
Processing Dataset: 1181it [01:06, 17.19it/s]
Processing Dataset: 1183it [01:07, 17.50it/s]
Processing Dataset: 1185it [01:07, 17.23it/s]
Processing Dataset: 1188it [01:07, 18.26it/s]
Processing Dataset: 1190it [01:07, 17.83it/s]
Processing Dataset: 1192it [01:07, 17.88it/s]
Processing Dataset: 1194it [01:07, 18.11it/s]
Processing Dataset: 1196it [01:07, 17.75it/s]
Processing Dataset: 1198it [01:07, 17.79it/s]
Processing Dataset: 1201it [01:08, 18.58it/s]
Processing Dataset: 1203it [01:08, 18.94it/s]
Processing Dataset: 1205it [01:08, 17.41it/s]
Processing Dataset: 1207it [01:08, 17.21it/s]
Processing Dataset: 1209it [01:08, 17.76it/s]
Processing Dataset: 1211it [01:08,

Processing Dataset: 1517it [01:26, 16.77it/s]
Processing Dataset: 1520it [01:26, 17.71it/s]
Processing Dataset: 1522it [01:26, 17.64it/s]
Processing Dataset: 1524it [01:26, 17.51it/s]
Processing Dataset: 1526it [01:26, 17.35it/s]
Processing Dataset: 1528it [01:27, 16.28it/s]
Processing Dataset: 1530it [01:27, 14.74it/s]
Processing Dataset: 1532it [01:27, 15.56it/s]
Processing Dataset: 1534it [01:27, 15.41it/s]
Processing Dataset: 1536it [01:27, 16.44it/s]
Processing Dataset: 1538it [01:27, 15.64it/s]
Processing Dataset: 1540it [01:27, 16.56it/s]
Processing Dataset: 1542it [01:27, 16.93it/s]
Processing Dataset: 1544it [01:28, 17.51it/s]
Processing Dataset: 1546it [01:28, 16.95it/s]
Processing Dataset: 1548it [01:28, 17.12it/s]
Processing Dataset: 1550it [01:28, 16.34it/s]
Processing Dataset: 1552it [01:28, 16.23it/s]
Processing Dataset: 1554it [01:28, 16.65it/s]
Processing Dataset: 1556it [01:28, 17.12it/s]
Processing Dataset: 1558it [01:28, 17.47it/s]
Processing Dataset: 1560it [01:29,

...

Processing Dataset: 7545it [07:05, 17.59it/s]
Processing Dataset: 7547it [07:05, 18.18it/s]
Processing Dataset: 7549it [07:05, 18.64it/s]
Processing Dataset: 7551it [07:05, 14.03it/s]
Processing Dataset: 7553it [07:05, 14.81it/s]
Processing Dataset: 7555it [07:05, 15.97it/s]
Processing Dataset: 7557it [07:06, 16.91it/s]
Processing Dataset: 7559it [07:06, 16.79it/s]
Processing Dataset: 7561it [07:06, 17.25it/s]
Processing Dataset: 7563it [07:06, 17.72it/s]
Processing Dataset: 7565it [07:06, 18.10it/s]
Processing Dataset: 7567it [07:06, 16.22it/s]
Processing Dataset: 7570it [07:06, 17.06it/s]
Processing Dataset: 7572it [07:06, 17.37it/s]
Processing Dataset: 7574it [07:07, 17.84it/s]
Processing Dataset: 7576it [07:07, 16.51it/s]
Processing Dataset: 7578it [07:07, 16.43it/s]
Processing Dataset: 7580it [07:07, 17.30it/s]
Processing Dataset: 7582it [07:07, 17.84it/s]
Processing Dataset: 7584it [07:07, 18.14it/s]
Processing Dataset: 7586it [07:07, 18.54it/s]
Processing Dataset: 7588it [07:07,

Processing Dataset: 7897it [07:24, 17.70it/s]
Processing Dataset: 7899it [07:25, 18.02it/s]
Processing Dataset: 7901it [07:25, 17.90it/s]
Processing Dataset: 7903it [07:25, 18.07it/s]
Processing Dataset: 7905it [07:25, 18.54it/s]
Processing Dataset: 7907it [07:25, 17.12it/s]
Processing Dataset: 7909it [07:25, 17.34it/s]
Processing Dataset: 7912it [07:25, 18.36it/s]
Processing Dataset: 7914it [07:25, 17.43it/s]
Processing Dataset: 7916it [07:26, 17.89it/s]
Processing Dataset: 7918it [07:26, 18.43it/s]
Processing Dataset: 7921it [07:26, 18.24it/s]
Processing Dataset: 7923it [07:26, 18.58it/s]
Processing Dataset: 7925it [07:26, 18.81it/s]
Processing Dataset: 7927it [07:26, 18.93it/s]
Processing Dataset: 7929it [07:26, 18.77it/s]
Processing Dataset: 7931it [07:26, 17.60it/s]
Processing Dataset: 7933it [07:26, 18.17it/s]
Processing Dataset: 7935it [07:27, 18.19it/s]
Processing Dataset: 7937it [07:27, 18.40it/s]
Processing Dataset: 7939it [07:27, 18.52it/s]
Processing Dataset: 7941it [07:27,

Processing Dataset: 8255it [07:45, 17.89it/s]
Processing Dataset: 8258it [07:45, 18.56it/s]
Processing Dataset: 8260it [07:45, 18.56it/s]
Processing Dataset: 8262it [07:45, 18.93it/s]
Processing Dataset: 8264it [07:45, 19.02it/s]
Processing Dataset: 8266it [07:45, 18.29it/s]
Processing Dataset: 8268it [07:45, 18.02it/s]
Processing Dataset: 8270it [07:45, 18.06it/s]
Processing Dataset: 8273it [07:46, 18.45it/s]
Processing Dataset: 8275it [07:46, 17.90it/s]
Processing Dataset: 8277it [07:46, 17.92it/s]
Processing Dataset: 8279it [07:46, 18.28it/s]
Processing Dataset: 8281it [07:46, 18.71it/s]
Processing Dataset: 8283it [07:46, 18.71it/s]
Processing Dataset: 8286it [07:46, 19.06it/s]
Processing Dataset: 8288it [07:46, 18.07it/s]
Processing Dataset: 8290it [07:46, 17.35it/s]
Processing Dataset: 8292it [07:47, 17.87it/s]
Processing Dataset: 8294it [07:47, 18.04it/s]
Processing Dataset: 8296it [07:47, 18.29it/s]
Processing Dataset: 8298it [07:47, 18.27it/s]
Processing Dataset: 8300it [07:47,

Processing Dataset: 8609it [08:05, 16.72it/s]
Processing Dataset: 8611it [08:05, 17.25it/s]
Processing Dataset: 8613it [08:05, 17.80it/s]
Processing Dataset: 8615it [08:05, 17.95it/s]
Processing Dataset: 8617it [08:05, 18.28it/s]
Processing Dataset: 8619it [08:05, 18.53it/s]
Processing Dataset: 8621it [08:05, 18.42it/s]
Processing Dataset: 8623it [08:05, 18.36it/s]
Processing Dataset: 8626it [08:06, 18.88it/s]
Processing Dataset: 8628it [08:06, 18.45it/s]
Processing Dataset: 8630it [08:06, 18.88it/s]
Processing Dataset: 8632it [08:06, 18.05it/s]
Processing Dataset: 8634it [08:06, 18.44it/s]
Processing Dataset: 8636it [08:06, 18.65it/s]
Processing Dataset: 8638it [08:06, 18.68it/s]
Processing Dataset: 8640it [08:06, 18.67it/s]
Processing Dataset: 8642it [08:06, 18.86it/s]
Processing Dataset: 8645it [08:07, 19.92it/s]
Processing Dataset: 8648it [08:07, 19.67it/s]
Processing Dataset: 8650it [08:07, 19.08it/s]
Processing Dataset: 8652it [08:07, 19.20it/s]
Processing Dataset: 8655it [08:07,

Processing Dataset: 8966it [08:25, 18.77it/s]
Processing Dataset: 8968it [08:25, 19.07it/s]
Processing Dataset: 8971it [08:25, 19.58it/s]
Processing Dataset: 8973it [08:25, 19.70it/s]
Processing Dataset: 8975it [08:25, 18.71it/s]
Processing Dataset: 8978it [08:25, 19.15it/s]
Processing Dataset: 8980it [08:25, 19.34it/s]
Processing Dataset: 8982it [08:25, 19.01it/s]
Processing Dataset: 8984it [08:25, 18.93it/s]
Processing Dataset: 8987it [08:26, 19.51it/s]
Processing Dataset: 8989it [08:26, 19.42it/s]
Processing Dataset: 8991it [08:26, 18.71it/s]
Processing Dataset: 8994it [08:26, 19.16it/s]
Processing Dataset: 8996it [08:26, 16.51it/s]
Processing Dataset: 8998it [08:26, 17.39it/s]
Processing Dataset: 9000it [08:26, 17.50it/s]
Processing Dataset: 9003it [08:26, 18.56it/s]
Processing Dataset: 9005it [08:27, 18.38it/s]
Processing Dataset: 9007it [08:27, 18.76it/s]
Processing Dataset: 9009it [08:27, 17.54it/s]
Processing Dataset: 9011it [08:27, 18.08it/s]
Processing Dataset: 9013it [08:27,

In [12]:
def create_loc_mask(tif, polys, evaluate=False):
    #Convert tif to PIL Image
    img = Image.fromarray(np.uint8(tif))
    if not evaluate:
        mask = Image.new("RGB", img.size, (0,0,0)) #concept-mode L is 8bit pixels, black and white
        draw = ImageDraw.Draw(mask)
        for poly in polys:
            x,y = poly.exterior.xy
            draw.polygon(list(zip(x,y)), fill=(255,255,255), outline=None)
    else:
        mask = Image.new("L", img.size, 0) #concept-mode L is 8bit pixels, black and white
        draw = ImageDraw.Draw(mask)
        for poly in polys:
            x,y = poly.exterior.xy
            draw.polygon(list(zip(x,y)), fill=1, outline=None)
    return np.asarray(mask)

In [15]:
a = [1,2]
b = [2,3,4]

In [16]:
c = a+b

In [25]:
images[0].split('images')[0]

'../datasets/geotiffs/tier1/'

In [1]:
import cv2

In [2]:
cv2.destroyAllWindows()